In [1]:
import IPython.display as ipd

import numpy as np
import torch


# import stft_64_pad_0 as stft
import stft_64 as stft
from audio_processing import griffin_lim

from scipy.io.wavfile import read
import time

In [2]:
hop_length = 2
frames = 6
win_length = 4*hop_length
channels = hop_length*2+1
wav_length = hop_length*frames


audio_origin = torch.rand((1,wav_length), dtype=torch.float64)
print(audio_origin.shape)
print(audio_origin[0,:10])

torch.Size([1, 12])
tensor([0.1643, 0.8396, 0.7345, 0.9361, 0.5169, 0.9671, 0.1016, 0.3702, 0.4463,
        0.2089], dtype=torch.float64)


In [3]:
# stft_fn = stft.STFT(filter_length=4, hop_length=1, win_length=4,
#                     window='hann')

stft_fn = stft.STFT(filter_length=win_length, hop_length=hop_length, win_length=win_length,
                    window=None)


def compare(a,b):
    return torch.mean(torch.abs(a-b)), torch.mean((a-b)*(a-b))

def compare_L1(ori,gen):
    return torch.mean(torch.abs(ori-gen)/torch.abs(ori))


def compare_L2(a,b):
    return torch.sum(torch.abs(a-b)), torch.sum((a-b)*(a-b))

def magnitude_to_L(magnitude, forward_basis):
    # magnitude:      [1, 2H+1, T]
    # forward_basis:  [2H+1, 1, 4H]
    
    # output L shape: [2H+1,T]
    H = (magnitude.shape[1]-1)//2
    basis = np.zeros((2*H+1,2*H+1), dtype= np.float64)
    basis[:, :] = forward_basis[:,0,:2*H+1]
    basis_inv = np.linalg.pinv(basis)
    L = np.matmul(basis_inv, magnitude[0,:,:]**2)
    L[1:2*H+1] = L[1:2*H+1]/2
    return L

In [4]:
magnitude, phase_origin = stft_fn.transform(audio_origin)

forward_basis = stft_fn.forward_basis
print(forward_basis.shape)
start_frame = 0
M_Rc = torch.sum(forward_basis[:channels,0,:] * audio_origin[:,start_frame: start_frame+win_length], dim =1)
M_Ic = torch.sum(forward_basis[channels:,0,:] * audio_origin[:,start_frame: start_frame+win_length], dim =1)
M_c_square = torch.sqrt(M_Rc**2+M_Ic**2)
print(M_c_square.shape)
print(magnitude.shape)
magnitude    = magnitude[:,:,2:-2]
phase_origin = phase_origin[:,:,2:-2]
print(magnitude[0,:,start_frame])
print(magnitude.shape)


torch.Size([10, 1, 8])
torch.Size([5])
torch.Size([1, 5, 7])
tensor([4.6303, 1.2648, 0.5239, 0.3511, 1.5957], dtype=torch.float64)
torch.Size([1, 5, 3])


In [5]:
for i in range(3):
    idx = 1
    sums = 0
    for t in range(i*hop_length, i*hop_length + win_length):
        sums = sums + idx * audio_origin[0,t]
        idx = idx*1
    print(sums)
    
    idx = 1
    sums = 0
    for t in range(i*hop_length, i*hop_length + win_length):
        sums = sums + idx * audio_origin[0,t]
        idx = idx*(-1)
    print(sums)

tensor(4.6303, dtype=torch.float64)
tensor(-1.5957, dtype=torch.float64)
tensor(4.2816, dtype=torch.float64)
tensor(-0.6829, dtype=torch.float64)
tensor(4.0746, dtype=torch.float64)
tensor(-0.6443, dtype=torch.float64)


In [14]:
L = magnitude_to_L(magnitude, stft_fn.forward_basis[:2*hop_length+1,:,:])
L = L.numpy()
print(L.shape)

for i in range(2**6):
    sgn10 = (i%(2**1))//(2**0)
    # sgn10 = 1 
    T10 = (sgn10 * 2 - 1) * np.sqrt(L[0,0] + 2*L[1,0] + 2*L[2,0] + 2*L[3,0] + 2*L[4,0])
    sgn20 = (i%(2**2))//(2**1)
    # sgn20 = 0
    T20 = (sgn20 * 2 - 1) * np.sqrt(L[0,0] - 2*L[1,0] + 2*L[2,0] - 2*L[3,0] + 2*L[4,0])
    S10 = (T10+T20)/2
    S20 = (T10-T20)/2
    
    sgn11 = (i%(2**3))//(2**2)
    # sgn11 = 1 
    T11 = (sgn11 * 2 - 1) * np.sqrt(L[0,1] + 2*L[1,1] + 2*L[2,1] + 2*L[3,1] + 2*L[4,1])
    sgn21 = (i%(2**4))//(2**3)
    # sgn21 = 0
    T21 = (sgn21 * 2 - 1) * np.sqrt(L[0,1] - 2*L[1,1] + 2*L[2,1] - 2*L[3,1] + 2*L[4,1])
    S11 = (T11+T21)/2
    S21 = (T11-T21)/2
    
    sgn12 = (i%(2**5))//(2**4)
    # sgn12 = 1 
    T12 = (sgn12 * 2 - 1) * np.sqrt(L[0,2] + 2*L[1,2] + 2*L[2,2] + 2*L[3,2] + 2*L[4,2])
    sgn22 = (i%(2**6))//(2**5)
    # sgn22 = 0
    T22 = (sgn22 * 2 - 1) * np.sqrt(L[0,2] - 2*L[1,2] + 2*L[2,2] - 2*L[3,2] + 2*L[4,2])
    S12 = (T12+T22)/2
    S22 = (T12-T22)/2
    
    x8_0 = S11 - S10
    x9_1 = S21 - S20
    x10_2 = S12 - S11
    x11_3 = S22 - S21
    
    A_matrix = np.zeros((10,8), dtype=np.double)
    b = np.zeros((10,1), dtype=np.double)
    
    
    ### equations for 0-4
    A_matrix[0,0] = 2*x8_0
    A_matrix[0,1] = 2*x9_1
    b[0,0] = L[0,1] - L[0,0] - x8_0**2 - x9_1**2
    
    
    A_matrix[1,1], A_matrix[1,7] = x8_0, x8_0
    A_matrix[1,0], A_matrix[1,2] = x9_1, x9_1
    b[1,0] = L[1,1] - L[1,0]-x9_1*x8_0
    
    A_matrix[2,2], A_matrix[2,6] = x8_0, x8_0
    A_matrix[2,3], A_matrix[2,7] = x9_1, x9_1
    b[2,0] = L[2,1] - L[2,0]
    
    A_matrix[3,3], A_matrix[3,5] = x8_0, x8_0
    A_matrix[3,4], A_matrix[3,6] = x9_1, x9_1
    b[3,0] = L[3,1] - L[3,0]
    
    A_matrix[4,4]= x8_0
    A_matrix[4,5]= x9_1
    b[4,0] = L[4,1] - L[4,0]
    
    ### equations for 5-9
    A_matrix[5,2] = 2*x10_2
    A_matrix[5,3] = 2*x11_3
    b[5,0] = L[0,2] - L[0,1] - x10_2**2 - x11_3**2
    
    
    A_matrix[6,1], A_matrix[6,3] = x10_2, x10_2
    A_matrix[6,2], A_matrix[6,4] = x11_3, x11_3
    b[6,0] = L[1,2] - L[1,1] - x10_2*x9_1 - x11_3*x10_2
    
    A_matrix[7,0], A_matrix[7,4] = x10_2, x10_2
    A_matrix[7,1], A_matrix[7,5] = x11_3, x11_3
    b[7,0] = L[2,2] - L[2,1] - x10_2*x8_0 - x11_3*x9_1
    
    A_matrix[8,7], A_matrix[8,5] = x10_2, x10_2
    A_matrix[8,0], A_matrix[8,6] = x11_3, x11_3
    b[8,0] = L[3,2] - L[3,1] - x11_3*x8_0
    
    A_matrix[9,6]= x10_2
    A_matrix[9,7]= x11_3
    b[9,0] = L[4,2] - L[4,1]
    
    temp = A_matrix.T @ A_matrix
    ans = (np.linalg.inv(temp)) @ (A_matrix.T @ b)
    b_hat = A_matrix @ ans
    
    test_arr = []
    for i in range(2*hop_length+1):
        test_arr.append(np.sum(ans[:8,0] * \
                               np.roll(ans[:8,0],i)
                              )
                       )
    test_arr[-1] = test_arr[-1]/2
    
    
    if np.sum(np.abs(test_arr - L[:,0]))<1e-6:
        print('###########')
        print(np.sum(np.abs(b_hat-b)))
        print('recon' , ans.T)
        print('origin', audio_origin[:,:8].numpy())

(5, 3)
###########
1.7381929229287607e-14
recon [[0.16431234 0.83960605 0.73453925 0.93608793 0.51688098 0.96711897
  0.10156159 0.37018266]]
origin [[0.16431234 0.83960605 0.73453925 0.93608793 0.51688098 0.96711897
  0.10156159 0.37018266]]
###########
1.7381929229287607e-14
recon [[-0.16431234 -0.83960605 -0.73453925 -0.93608793 -0.51688098 -0.96711897
  -0.10156159 -0.37018266]]
origin [[0.16431234 0.83960605 0.73453925 0.93608793 0.51688098 0.96711897
  0.10156159 0.37018266]]
